# HW5

For this homework, we are going to work with [*Indoor User Movement Prediction from RSS data*](https://archive.ics.uci.edu/ml/datasets/Indoor+User+Movement+Prediction+from+RSS+data) dataset from UCI.  The homework is due Friday, December 21st midnight. 

## Task 1

Download the dataset and unzip it in under a subdirectory of `data` folder named `rss_data`.

The files we are interested is in the subfolder `dataset`.  Each of these files whose names that start with `MovementAAL_RSS_` contain data collected from indivuduals. Each of these files represent a single data point.  There are 314 of these files, and hence, you have 314 data points.  Each file has 4 columns but the number of rows change from file to file.  

There is also a file named `MovementALL_target.csv` in that folder. This file tells us the class each of these measurements are assigned. Some of these measurements are labelled with +1 and some are labelled with -1.

## Task 2

Construct a SVM model that separates +1 labelled data points from -1 data points.  You must first solve the problem that these datapoints do not have the same number of rows even though they all have the same number of columns. 

## Task 3

Using [Keras](https://keras.io/getting-started/sequential-model-guide/) write a neural network model that separates +1 labelled data points from -1 data points.

## Notes

1. You must document each step of your tasks: what are you doing, why are you doing it, what problems you encountered and how you solved it.  All of these must be explained and documented.  Solutions without sufficient documentations will be penalized accordingly. 50% of your points will come from your code, while the other 50% will come from your explanations.

1. You can use MS Excel to inspect the files, but loading them up to python using pandas and inspecting them there under jupyter is easier.

3. Put the data in a separate subfolder of your `data` folder and rename it `rss_data`. I'll take points off if the data is not saved under the correct place.

1. For both of Task 2 and Task 3, you must split your data into a train and test set, and then evaluate the accuracy of your model on the test set.




# TASK 1:

Data yı hazırlıyoruz

In [1]:
import glob
import pandas as pd
import numpy as np

Gerekli kütüphaneler yüklendi

In [2]:
csv_files=glob.glob("..//data//rss_data//dataset//MovementAAL_RSS_*.csv")
frames=[]
for csv in csv_files:
    df=pd.read_csv(csv)
    frames.append(df)

glob kütüphanesinin glob fonksiyonuyla 314 adet datayı csv_files listesine atadık for döngüsüyle de frames listesine attık

In [3]:
a=100
for i in frames:
    a=min(a,len(i))
print(a)

19


Datalar eşit değil en düşük olan sayıdakini bulup sondan kesicez son bulunduğu lokasyonu tespit için

In [4]:
l=[]
for i in frames:
    for row in range(1,20):
        for col in range(4):
            l.append(i.iloc[-row,col])

tüm datayı düz liste yapıp tekrar şekillendirip dataframe haline getiriyorum

In [5]:
k=np.array(l)

In [6]:
k=k.reshape(314,76)

In [7]:
df=pd.DataFrame(k)

target datasını çekiyorum hangi odada fln olduğunu gösteren data ,(314,2) geldiği için sequence kolonunu atıyorum

In [8]:
targets = pd.read_csv('..//data//rss_data//dataset//MovementAAL_target.csv')
targets=targets.drop("#sequence_ID",axis=1)

In [9]:
df=np.column_stack([df,targets])

In [10]:
df=pd.DataFrame(df)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
0,-0.14286,-0.60,-0.28571,-0.10,0.00000,0.04,-0.047619,-0.05,0.00000,0.04,...,0.40,-0.047619,0.04,-0.095238,0.05,-0.095238,-0.08,0.142860,-0.55,1.0
1,0.19048,0.04,-0.38095,0.20,0.19048,0.00,-0.190480,-0.25,0.14286,-0.04,...,0.10,0.000000,-0.12,0.142860,0.10,-0.285710,-0.24,0.095238,0.15,1.0
2,0.28571,0.56,-0.57143,-0.75,0.33333,0.40,-0.952380,-0.65,0.14286,0.56,...,-0.70,0.428570,0.04,-0.619050,-0.70,0.047619,0.32,-0.714290,-0.30,1.0
3,0.28571,0.20,-1.00000,-0.70,0.00000,0.04,-1.000000,-0.55,0.00000,0.04,...,-0.65,0.523810,0.60,-0.428570,-0.55,0.476190,0.32,-0.380950,-0.30,1.0
4,0.42857,0.68,-0.28571,-0.30,0.42857,0.76,-0.476190,-0.60,0.00000,0.36,...,-1.00,0.285710,0.60,-0.571430,-1.00,0.142860,0.76,-0.666670,-0.55,1.0


dataframe haline getirilirdi

data hazırlandı.

# Task 2:

Svm kütüphanelerini alma datayı bölme işlemleri aşağıda rfb en etkili olduğundan rfb yi seçtim gamma bir çok deneme sonrası optimal burda kaldı

In [42]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score


In [43]:
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,:76],df.iloc[:,76])
sv=SVC(kernel="rbf",gamma=0.03,C=2)
sv.fit(X_train,y_train)
predicted=sv.predict(X_test)
print(accuracy_score(y_test,predicted))
print(confusion_matrix(y_test,predicted))

0.7468354430379747
[[29 11]
 [ 9 30]]


data sonucu yeterli değil son zamanlarda machine learningin herkesin ağzında sakız olmasının sebebi olan neural network te daha başarılı olucak muhtemelen

# TASK 3:

Keras kütüphanelerini atma

In [55]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing import sequence

In [54]:
model = Sequential()
model.add(Embedding(76, 314))
model.add(LSTM(314, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation='relu', input_dim=76))
model.add(LSTM(314, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10, activation='sigmoid'))
model.add(LSTM(314, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='sigmoid'))
model.add(LSTM(314, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='relu'))
model.add(LSTM(314, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='sigmoid'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

ValueError: Input 0 is incompatible with layer lstm_10: expected ndim=3, found ndim=2

In [49]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test,y_test))

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: x=           0         1         2         3         4         5         6   \
285  0.142860  0.440000 -0.857140 -0.450000  0.238100  0.040000 -0.714290   
203 -0.545450 -0.777780  0.733330  0.276600 -0.545450 -0.600000  0.466670   
169  0.000000  1.000000 -0.600000 -0.489360  0.136360  0.911110 -0.644440   
99  -0.288890 -0.794870  0.681820  0.857140 -0.288890 -0.743590  0.772730   
189 -0.380950 -0.040000  0.142860  0.050000 -0.428570  0.080000  0.142860   
279  0.142860 -0.120000 -0.285710 -0.250000  0.000000 -0.040000 -0.285710   
41  -0.333330 -0.538460 -0.818180 -0.904760 -0.333330  0.384620 -0.818180   
33   1.000000  0.025641 -0.727270 -0.571430  1.000000 -0.025641 -0.590910   
45   0.095238 -0.280000 -0.095238  0.250000  0.142860 -0.160000 -0.142860   
284  0.571430  0.360000 -0.571430 -0.700000  0.714290  0.440000 -0.428570   
193 -0.454550 -0.777780  0.288890  0.148940 -0.318180

In [51]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(76, output_dim=76))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=16, epochs=10)
score = model.evaluate(X_test, y_test, batch_size=16)

Epoch 1/10


InvalidArgumentError: indices[2,33] = -1 is not in [0, 76)
	 [[Node: embedding_5/embedding_lookup = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@embedding_5/embeddings"], validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_5/embeddings/read, embedding_5/Cast)]]

Caused by op 'embedding_5/embedding_lookup', defined at:
  File "C:\Users\gurbu\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\gurbu\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\gurbu\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\gurbu\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\gurbu\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\gurbu\Anaconda3\lib\site-package